In [90]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [91]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [92]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [93]:
from google.colab import files
uploaded=files.upload()

In [110]:
df=spark.read.format("csv").load("movies.csv")
df.show()

+--------------------+--------------------+----+
|                 _c0|                 _c1| _c2|
+--------------------+--------------------+----+
|1::Toy Story (199...|                null|null|
|2::Jumanji (1995)...|                null|null|
|3::Grumpier Old M...|                null|null|
|4::Waiting to Exh...|                null|null|
|5::Father of the ...|                null|null|
|6::Heat (1995)::A...|                null|null|
|7::Sabrina (1995)...|                null|null|
|8::Tom and Huck (...|                null|null|
|9::Sudden Death (...|                null|null|
|10::GoldenEye (19...|                null|null|
|11::American Pres...| The (1995)::Come...|null|
|12::Dracula: Dead...|                null|null|
|13::Balto (1995):...|                null|null|
|14::Nixon (1995):...|                null|null|
|15::Cutthroat Isl...|                null|null|
|16::Casino (1995)...|                null|null|
|17::Sense and Sen...|                null|null|
|18::Four Rooms (1..

In [111]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema=StructType([\
                  StructField("Id",IntegerType(),True),\
                  StructField("Movie_name",StringType(),True),\
                  StructField("Movie_genre",StringType(),True)\
                   ])

In [112]:
df=spark.read.csv("movies.csv",sep="::",schema=schema)
df.show()

+---+--------------------+--------------------+
| Id|          Movie_name|         Movie_genre|
+---+--------------------+--------------------+
|  1|    Toy Story (1995)|Animation|Childre...|
|  2|      Jumanji (1995)|Adventure|Childre...|
|  3|Grumpier Old Men ...|    Comedy|Romance,,|
|  4|Waiting to Exhale...|      Comedy|Drama,,|
|  5|Father of the Bri...|            Comedy,,|
|  6|         Heat (1995)|Action|Crime|Thri...|
|  7|      Sabrina (1995)|    Comedy|Romance,,|
|  8| Tom and Huck (1995)|Adventure|Childre...|
|  9| Sudden Death (1995)|            Action,,|
| 10|    GoldenEye (1995)|Action|Adventure|...|
| 11|American Presiden...|Comedy|Drama|Roma...|
| 12|Dracula: Dead and...|     Comedy|Horror,,|
| 13|        Balto (1995)|Animation|Childre...|
| 14|        Nixon (1995)|             Drama,,|
| 15|Cutthroat Island ...|Action|Adventure|...|
| 16|       Casino (1995)|    Drama|Thriller,,|
| 17|Sense and Sensibi...|     Drama|Romance,,|
| 18|   Four Rooms (1995)|          Thri

In [113]:
schema=StructType([\
                  StructField("Id",IntegerType(),True),\
                  StructField("Movie_name",StringType(),True),\
                  StructField("Movie_genre",StringType(),True),\
                  StructField("Year_released",IntegerType(),True)\
                   ])

In [114]:
df=spark.read.csv("movies.csv",sep="::",schema=schema)
df.show()

+---+--------------------+--------------------+-------------+
| Id|          Movie_name|         Movie_genre|Year_released|
+---+--------------------+--------------------+-------------+
|  1|    Toy Story (1995)|Animation|Childre...|         null|
|  2|      Jumanji (1995)|Adventure|Childre...|         null|
|  3|Grumpier Old Men ...|    Comedy|Romance,,|         null|
|  4|Waiting to Exhale...|      Comedy|Drama,,|         null|
|  5|Father of the Bri...|            Comedy,,|         null|
|  6|         Heat (1995)|Action|Crime|Thri...|         null|
|  7|      Sabrina (1995)|    Comedy|Romance,,|         null|
|  8| Tom and Huck (1995)|Adventure|Childre...|         null|
|  9| Sudden Death (1995)|            Action,,|         null|
| 10|    GoldenEye (1995)|Action|Adventure|...|         null|
| 11|American Presiden...|Comedy|Drama|Roma...|         null|
| 12|Dracula: Dead and...|     Comedy|Horror,,|         null|
| 13|        Balto (1995)|Animation|Childre...|         null|
| 14|   

In [118]:
#df[['Movie_name','Year_released']]= df.Movie_name.str.extract('(.)\s\((.\d+)', expand=True)
#df[['Movie_name','Year_released']] = df.Movie_name.str.split('\(|\)', expand=True).iloc[:,[0,1]]
'''from pyspark.sql.functions import split 
df=df.strip("Year_released").str.strip("()")
df.show()'''
#df['Year_released'] = df.Movie_name.str.strip('()')[0]
#res = df['Movie_name'].str.extract(r'\((.*?)\)')
#print(res)
#df['Movie_name'] = df['Movie_name'].str.extract(r"\(\$(.*)\)")
#print (df)
from pyspark.sql.functions import split
df1 = df.withColumn('Year_released', split(df['Movie_name'], ",").getItem(0)) 
df1.show(truncate=False)

+---+-------------------------------------+------------------------------+-------------------------------------+
|Id |Movie_name                           |Movie_genre                   |Year_released                        |
+---+-------------------------------------+------------------------------+-------------------------------------+
|1  |Toy Story (1995)                     |Animation|Children's|Comedy,, |Toy Story (1995)                     |
|2  |Jumanji (1995)                       |Adventure|Children's|Fantasy,,|Jumanji (1995)                       |
|3  |Grumpier Old Men (1995)              |Comedy|Romance,,              |Grumpier Old Men (1995)              |
|4  |Waiting to Exhale (1995)             |Comedy|Drama,,                |Waiting to Exhale (1995)             |
|5  |Father of the Bride Part II (1995)   |Comedy,,                      |Father of the Bride Part II (1995)   |
|6  |Heat (1995)                          |Action|Crime|Thriller,,       |Heat (1995)           

In [121]:
#df['Year_released'] = df['Year_released'].str.strip('(',")").astype(int)
#df['Year_released'].str.replace(r"\(.*\)","")

TypeError: ignored

In [100]:
#from pyspark.sql import functions as F
#df.withColumn("Year_released ", F.regexp_replace(F.regexp_replace(F.regexp_replace("Year_released ", "(", "-"), "", ""), ")", "")).show()

In [101]:

#df['Year_released'] = df.withColumn['Year_released'].str.replace(r"(","")
#df['Year_released'] = df.withColumn['Year_released'].str.replace(r")","")

In [107]:
from pyspark.sql.functions import regexp_replace
df = df.select("Year_released", regexp_replace("Year_released", "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ", ""
                                    ).alias('Year_released'))


AnalysisException: ignored

In [108]:
df.show

<bound method DataFrame.show of +-------------+-------------+
|Year_released|Year_released|
+-------------+-------------+
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
|         null|         null|
+-------------+-------------+
only showing top 20 rows
>

In [103]:
#8
from google.colab import files
uploaded=files.upload()

In [21]:
schema=StructType([\
                  StructField("Id",IntegerType(),True),\
                  StructField("gender",StringType(),True),\
                  StructField("MovieID",IntegerType(),True),\
                  StructField("rating",IntegerType(),True),\
                  StructField("numbers",IntegerType(),True)\
                   ])

In [33]:
df=spark.read.csv("users.csv",sep="::",schema=schema)
df.show()

+----+------+-------+------+-------+
|  Id|gender|MovieID|rating|numbers|
+----+------+-------+------+-------+
|2022|  null|   null|  null|   null|
|null|  null|   null|  null|   null|
|null|  null|   null|  null|   null|
|   1|     F|      1|    10|  48067|
|   2|     M|     56|    16|  70072|
|   3|     M|     25|    15|  55117|
|   4|     M|     45|     7|   2460|
|   5|     M|     25|    20|  55455|
|   6|     F|     50|     9|  55117|
|   7|     M|     35|     1|   6810|
|   8|     M|     25|    12|  11413|
|   9|     M|     25|    17|  61614|
|  10|     F|     35|     1|  95370|
|  11|     F|     25|     1|   4093|
|  12|     M|     25|    12|  32793|
|  13|     M|     45|     1|  93304|
|  14|     M|     35|     0|  60126|
|  15|     M|     25|     7|  22903|
|  16|     F|     35|     0|  20670|
|  17|     M|     50|     1|  95350|
+----+------+-------+------+-------+
only showing top 20 rows



In [24]:
df=df.drop("numbers")
df.show()

+----+------+-------+------+
|  Id|gender|MovieID|rating|
+----+------+-------+------+
|2022|  null|   null|  null|
|null|  null|   null|  null|
|null|  null|   null|  null|
|   1|     F|      1|    10|
|   2|     M|     56|    16|
|   3|     M|     25|    15|
|   4|     M|     45|     7|
|   5|     M|     25|    20|
|   6|     F|     50|     9|
|   7|     M|     35|     1|
|   8|     M|     25|    12|
|   9|     M|     25|    17|
|  10|     F|     35|     1|
|  11|     F|     25|     1|
|  12|     M|     25|    12|
|  13|     M|     45|     1|
|  14|     M|     35|     0|
|  15|     M|     25|     7|
|  16|     F|     35|     0|
|  17|     M|     50|     1|
+----+------+-------+------+
only showing top 20 rows

